In [3]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
tickers = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'META', 'TSLA', 'BRK-B', 'JPM', 'V']
start_date = '2010-01-01'
end_date = '2024-12-31'
train_end = '2023-12-31'
test_start = '2024-01-01'

# --- Download Data ---
data = yf.download(tickers, start=start_date, end=end_date)

# --- Extract Close prices and create LogClose ---
close_data = data['Close'].copy()
log_close_data = np.log(close_data)
log_data = pd.concat([close_data, log_close_data], axis=1, keys=['Close', 'LogClose'])

# --- Feature Engineering ---
max_lag = 5
feature_df = pd.DataFrame(index=log_data['LogClose'].index)

for ticker in tickers:
    for lag in range(1, max_lag + 1):
        feature_df[f"{ticker}_lag{lag}"] = log_data['LogClose'][ticker].shift(lag)

# --- Training and Evaluation ---
results_rf = {}

for ticker in tickers:
    print(f"\n🌲 Training Random Forest for {ticker}")
    
    # Define target: next-day log return
    feature_df[f"target_{ticker}"] = log_data['LogClose'][ticker].shift(-1) - log_data['LogClose'][ticker]
    
    # Clean NaNs
    model_data = feature_df.dropna()
    
    # Time-based split
    train_data = model_data.loc[:train_end]
    test_data = model_data.loc[test_start:]
    
    X_train = train_data.drop(columns=[f"target_{ticker}"])
    y_train = train_data[f"target_{ticker}"]
    X_test = test_data.drop(columns=[f"target_{ticker}"])
    y_test = test_data[f"target_{ticker}"]
    
    # Fit Random Forest
    rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = rf_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"✅ {ticker} RMSE: {rmse:.6f}")
    
    results_rf[ticker] = {
        'model': rf_model,
        'rmse': rmse,
        'y_test': y_test,
        'y_pred': pd.Series(y_pred, index=y_test.index)
    }


C:\Users\joelt\AppData\Local\Temp\ipykernel_52228\302798680.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  10 of 10 completed



🌲 Training Random Forest for AAPL
✅ AAPL RMSE: 0.015345

🌲 Training Random Forest for MSFT
✅ MSFT RMSE: 0.011386

🌲 Training Random Forest for NVDA
✅ NVDA RMSE: 0.030605

🌲 Training Random Forest for AMZN
✅ AMZN RMSE: 0.017570

🌲 Training Random Forest for GOOGL
✅ GOOGL RMSE: 0.015778

🌲 Training Random Forest for META
✅ META RMSE: 0.023467

🌲 Training Random Forest for TSLA
✅ TSLA RMSE: 0.038728

🌲 Training Random Forest for BRK-B
✅ BRK-B RMSE: 0.009425

🌲 Training Random Forest for JPM
✅ JPM RMSE: 0.012884

🌲 Training Random Forest for V
✅ V RMSE: 0.012455
